In [1]:
import cPickle as pickle
from bs4 import BeautifulSoup
import nltk
from nltk.stem.porter import *
import numpy as np
import sys
import os.path
sys.path.append(os.path.join('..', 'TFIDF'))
sys.path.append(os.path.join('..', 'util'))
from timer import Timer

In [ ]:
from HTMLParser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [8]:
questions = pickle.load(open("lstOfQuestions.pkl", "rb"))
questions = questions[:10000]

words_dict = {}
question_texts = []
total = len(questions)
count = 0
for question in questions :
    if count % (total / 10) == 0:
        print count
    VALID_TAGS = ['p']

    soup = BeautifulSoup(question['body'])

    VALID_TAGS = ['p']
    INVALID_TAGS = ['code', 'a']

    text = ' '.join([s.get_text() for s in soup('p')])

    tokens = nltk.word_tokenize(text)

    stemmer = PorterStemmer()

    words = [stemmer.stem(token.encode('ascii', 'ignore').lower()) for token in tokens]

    question_texts.append(words)
    
    for word in words :
        if word in words_dict :
            words_dict[word] += 1
        else :
            words_dict[word] = 1
    
    count += 1
    
id_hash = {}
count = 0
for question in questions:
    if question['question_id'] not in id_hash:
        id_hash[question['question_id']] = 1
        count += 1

print count

# Go through dictionary, removing entries that have a value less
# than delta
delta = 2
words_dict2 = {key: value for key, value in words_dict.items()
                 if value >= delta }

word_indices = {}
index = 0
for word in words_dict2 :
    word_indices[word] = index
    index += 1

word_indices = list(word_indices.iteritems())

output = open('wordIndices_md.pkl', 'wb')
pickle.dump(word_indices, output, -1)
output.close()

output = open('questionTexts_md.pkl', 'wb')
pickle.dump(question_texts, output, -1)
output.close()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [10]:
try:
     tfidf = reload(tfidf)
except NameError:
    import TFIDF_numpy as tfidf

try :
    word_indices
except NameError:
    word_indices = pickle.load(open('wordIndices_md.pkl', 'rb'))

try :
    question_texts
except NameError:
    question_texts = pickle.load(open('questionTexts_md.pkl', 'rb'))
    
tfidf.init_globals()
tfidf.load_questions(question_texts)
tfidf.load_indices(word_indices)
tfidf.init_tfs()
serial_tfs = tfidf.create_tfs()
serial_idf = tfidf.calculate_idf()
serial_tfidfs = tfidf.calculate_tfidfs()
# tfidf.calculate_tfidf_norms()
# tfidf.calculate_cossim(question_texts[0])
serial_simhashes = tfidf.calculate_simhashes()

Time for initialization: 0.000574111938477
Time for load_questions: 1.19209289551e-06
Time for load_indices: 0.0149481296539
Time for init_tfs: 1.35794305801
Time for create_tfs: 1.69611787796
Time for create_idf: 1.34325480461
Time for calculate_tfidfs: 0.650036096573
Time for calculate_simhashes: 85.1130979061


In [11]:
try :
    tfidf_c = reload(tfidf_c)
except :
    import TFIDF_cython as tfidf_c

try :
    word_indices
except NameError:
    word_indices = pickle.load(open('wordIndices_sm.pkl', 'rb'))

try :
    question_texts
except NameError:
    question_texts = pickle.load(open('questionTexts_sm.pkl', 'rb'))

# Preprocess for cython code
tfidf_c.init_globals(1, True, "coarse", 64)
tfidf_c.load_questions(question_texts)
tfidf_c.load_indices(word_indices)
tfidf_c.init_tfs()
cython_tfs = np.asarray(tfidf_c.create_tfs())[:,0:len(word_indices)]
cython_idf = np.asarray(tfidf_c.calculate_idf(len(word_indices)))[0:len(word_indices)]
tfidf_c.init_tfidfs()
cython_tfidfs = np.asarray(tfidf_c.calculate_tfidfs())[:,0:len(word_indices)]
cython_simhashes = np.asarray(tfidf_c.calculate_simhashes())

Compiling with /usr/local/bin/clang-omp
Time for Initialization: 3.09944152832e-05
Time for load_questions: 0.0709958076477
Time for load_indices: 0.0325329303741
Time for init_tfs: 0.963369846344
Time for create_tfs: 0.265480041504
Time for calculate_idf: 0.206746816635
Time for int_tfidfs: 0.926871061325
Using AVX
Time for calculate_tfidfs: 0.148722171783
Time for calculate_simhashes: 0.303247928619


In [12]:
# Verification
print np.linalg.norm(cython_tfs - serial_tfs)
print np.linalg.norm(cython_idf - serial_idf)
print np.linalg.norm(cython_tfidfs - serial_tfidfs)

0.0
3.05905382673e-05
0.000547179457161


In [49]:
#The following are from https://yesteapea.wordpress.com/2013/03/03/counting-the-number-of-set-bits-in-an-integer/
def numBits64(i):
    i = i - ((i >> np.uint64(1)) & np.uint64(0x5555555555555555))
    i = (i & np.uint64(0x3333333333333333)) + ((i >> np.uint64(2)) & np.uint64(0x3333333333333333))
    i = ((i + (i >> np.uint64(4))) & np.uint64(0x0F0F0F0F0F0F0F0F))
    return (i*(np.uint64(0x0101010101010101)))>>np.uint64(56)

def numBits32(i):
    i = i - ((i >> 1) & 0x55555555)
    i = (i & 0x33333333) + ((i >> 2) & 0x33333333)
    i = ((i + (i >> 4)) & 0x0F0F0F0F)
    return (i*(0x01010101))>>24

In [50]:
distances = [[0 for i in xrange(len(cython_simhashes))] for j in xrange(len(cython_simhashes))]
for i in xrange(len(cython_simhashes)):
    for j in xrange(len(cython_simhashes)):
        distances[i][j] = numBits64(cython_simhashes[i] ^ cython_simhashes[j])
    if i % 1000 == 0:
        print (float(i)/len(cython_simhashes))*100, " percent done"

0.0  percent done
10.0  percent done
20.0  percent done
30.0  percent done
40.0  percent done
50.0  percent done
60.0  percent done
70.0  percent done
80.0  percent done
90.0  percent done


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: RuntimeWarning: overflow encountered in ulong_scalars


In [51]:
from sklearn.cluster import DBSCAN

In [52]:
db = DBSCAN(metric="precomputed", eps=2, min_samples=3).fit(distances)

In [16]:
sum(map(lambda x: 1 if x >= 0 else 0, db.labels_))

9994

In [26]:
all_one_labels = []
counter = 0
for i in xrange(len(db.labels_)):
    if db.labels_[i] == -1:
        counter += 1
    elif db.labels_[i] == 4:
        all_one_labels.append(i)
print counter
print all_one_labels

6
[]


In [ ]:
for i in all_one_labels: 
    print " ".join(question_texts[i])
    print ""